# Testing

`soupsavvy` provides utilities for testing your selectors. By leveraging the features of the `soupsavvy.testing` subpackage, you can validate your selectors, check them against various edge cases, and ensure they work as expected.

## Generators

The subpackage includes HTML code generators designed specifically for testing purposes. These generators help you create controlled HTML structures to simulate various scenarios, ensuring your selectors perform as expected.

### Attribute Generator

Attribute generator is used to generate string representation of html attributes. While not so useful on its own, it becomes valuable when combined with the `TagGenerator`, offering extensive customization options.

#### Empty Attribute

If only the first parameter (the attribute name) is passed to the `AttributeGenerator`, it generates an attribute with an empty value.

In [ ]:
from soupsavvy.testing import AttributeGenerator

generator = AttributeGenerator("class")
generator.generate()

#### Constant value

By passing the `value` parameter, you can set a specific value for the attribute.

In [ ]:
from soupsavvy.testing import AttributeGenerator

generator = AttributeGenerator("class", value="book")
generator.generate()

### Templates

Templates add another layer of customization by generating strings based on predefined logic, useful for creating dynamic and varied content in your test HTML.

#### ChoiceTemplate

The `ChoiceTemplate` allows you to generate a string by randomly selecting from a provided list of strings. For reproducibility, the `seed` parameter can be set to ensure the same output is generated across multiple runs.

In [ ]:
from soupsavvy.testing import AttributeGenerator, ChoiceTemplate

template = ChoiceTemplate(["book", "article", "blog"], seed=42)
generator = AttributeGenerator("class", value=template)
generator.generate()

#### RandomTemplate

The `RandomTemplate` generates a string from randomly selected characters. The `length` parameter defines the string length (default is 4). Like `ChoiceTemplate`, the `seed` parameter ensures consistent output if needed.

In [ ]:
from soupsavvy.testing import AttributeGenerator, RandomTemplate

template = RandomTemplate(length=5, seed=42)
generator = AttributeGenerator("class", value=template)
generator.generate()

#### User-defined Templates

For advanced customization, you can create your own templates by subclassing `soupsavvy.testing.BaseTemplate` and implementing the `generate` method to return a string based on your specific logic.

Here’s how you can define a custom template:

In [ ]:
from soupsavvy.testing import BaseTemplate, TagGenerator


class CustomTemplate(BaseTemplate):
    def __init__(self, connection): ...

    def generate(self):
        # connects to external service
        result = "Hello from somewhere!"
        return result


template = CustomTemplate(connection=None)
generator = TagGenerator("span", text=template)
generator.generate()

### TagGenerator

`TagGenerator` is the primary tool for generating single HTML tags with customizable attributes, text, and child elements.

#### Name

The `name` parameter is required and specifies the tag name, such as `div`, `span`, or `p`.

In [ ]:
from soupsavvy.testing import TagGenerator

generator = TagGenerator("div")
generator.generate()

#### Attributes

The `attrs` parameter allows you to define the attributes of the tag. It accepts an iterable containing:

- `str`: Just the attribute name, resulting in an empty value.
- `tuple`: A pair where the first element is the attribute name and the second is the value.
- `AttributeGenerator`: An object that dynamically generates attribute values.

All attributes are converted to `AttributeGenerator` objects, enabling the use of templates for dynamic generation. Duplicate attributes will raise an error.

In [ ]:
from soupsavvy.testing import AttributeGenerator, RandomTemplate, TagGenerator

attrs = (
    "href",
    ("class", "link"),
    ("data-id", RandomTemplate(seed=42)),
    AttributeGenerator("title", value="buy"),
)
generator = TagGenerator("a", attrs=attrs)
generator.generate()

Attributes of the tag must be unique, so trying to define `TagGenerator` with duplicate attributes will raise an error.

In [ ]:
import soupsavvy.exceptions as exc
from soupsavvy.testing import TagGenerator

try:
    generator = TagGenerator("a", attrs=["href", "href"])
except exc.SoupsavvyException as e:
    print(type(e))
    print(e)

#### Children

The `children` parameter lets you specify the tag's children, which must be `TagGenerator` objects. If no children are specified, the tag is created without any.

In [ ]:
from soupsavvy.testing import TagGenerator

child_generator = TagGenerator("span")
generator = TagGenerator(
    "div",
    attrs=["class"],
    children=[child_generator],
)
generator.generate()

#### Self-closing Tags

Self-closing tags like `br` are automatically handled. Defining a self-closing tag with children will raise an error.

In [ ]:
from soupsavvy.testing import TagGenerator

generator = TagGenerator("br")
generator.generate()

In [ ]:
from soupsavvy.testing import TagGenerator

try:
    generator = TagGenerator("hr", children=[TagGenerator("span")])
except exc.SoupsavvyException as e:
    print(type(e))
    print(e)

#### Text

The `text` parameter allows you to add text content to the tag. This can be a static string or dynamically generated using templates.

In [ ]:
from soupsavvy.testing import TagGenerator

generator = TagGenerator("span", text="Hello, World!")
generator.generate()

In [ ]:
from soupsavvy.testing import ChoiceTemplate, TagGenerator

template = ChoiceTemplate(["Hello, World!", "Hello, blog!"], seed=42)
generator = TagGenerator("span", text=template)
generator.generate()

### Usage

Let's see how to use these generators in practice. In this example, we'll test a selector targeting `span` elements with text content starting with "Hello" that are children of `div` elements with `class` attribute of `book` and `role` attribute of `section`.

In [ ]:
import re

from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, ClassSelector, PatternSelector, TypeSelector
from soupsavvy.testing import AttributeGenerator, ChoiceTemplate, TagGenerator

# 1: define the generator
template = ChoiceTemplate(["Hello, World!", "Hello, blog!"], seed=42)
child_generator = TagGenerator("span", text=template)
generator = TagGenerator(
    "div",
    attrs=[
        AttributeGenerator("class", value="book"),
        AttributeGenerator("role", value="section"),
    ],
    children=[child_generator],
)

# 2: define the selector
selector = (
    TypeSelector("div")
    & ClassSelector("book")
    & AttributeSelector("role", value="section")
) > (TypeSelector("span") & PatternSelector(re.compile(r"^Hello")))

# 3: generate the soup
text = generator.generate()
soup = BeautifulSoup(text, features="lxml")

# 4: test selector on generated soup
selector.find(soup)

In this example, we dynamically generate the HTML content using `TagGenerator` and then checked if the selector correctly identifies the intended elements.

## Conclusion

By using these generators, you can easily create HTML code to test whether your `soupsavvy` selector is correctly defined. This approach enables you to validate complex selectors in a dynamic and controlled environment.

**Enjoy `soupsavvy` and leave us feedback!**  
**Happy scraping!**